In [410]:
import pandas as pd
import random
import re       # regular expression library
from sklearn.feature_extraction.text import CountVectorizer


In [411]:
csv_file = 'money.csv'                              # the input csv file
topic_file = 'sklearn_topics_'+csv_file                     # the output csv file containing the content above with associated topics

In [412]:
data_path = 'data/'+csv_file
topic_path = 'topics/'+topic_file
df = pd.read_csv(data_path)
df.head()


,text
0,"Let’s say I had an apple, and you had an orang..."
1,an apple.
2,"We could trade, and both of us would end up ha..."
3,"But now let’s say I had an apple tree, and you..."
4,"the apple tree were ready, but the oranges wou..."


In [413]:
# Remove punctuation
df['processed_text'] = \
df['text'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
df['processed_text'] = \
df['processed_text'].map(lambda x: x.lower())

df = df.drop(columns='text')

# Print out the first rows of papers
df.head()

<>:3: DeprecationWarning: invalid escape sequence '\.'
<>:3: DeprecationWarning: invalid escape sequence '\.'
/var/folders/4z/4w7rz93j1gdcqnmppw47r0jr0000gn/T/ipykernel_906/3455528102.py:3: DeprecationWarning: invalid escape sequence '\.'
  df['text'].map(lambda x: re.sub('[,\.!?]', '', x))


,processed_text
0,let’s say i had an apple and you had an orange...
1,an apple
2,we could trade and both of us would end up hap...
3,but now let’s say i had an apple tree and you ...
4,the apple tree were ready but the oranges woul...


In [414]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
dtm = cv.fit_transform(df['processed_text'])

LDA = LatentDirichletAllocation(n_components=3,random_state=42)
LDA.fit(dtm)

LatentDirichletAllocation(n_components=3, random_state=42)

In [415]:
for i in range(10):
    random_word_id = random.randint(0,lim)
    print(cv.get_feature_names_out()[random_word_id])

for i in range(10):
    random_word_id = random.randint(0,lim)
    print(cv.get_feature_names_out()[random_word_id])

american
moving
correspondent
communicated
works
1900s
breaks
institutions
came
used
shipping
allowed
11500
day
correspondent
didn
financial
institutions
era
easy


In [416]:
single_topic = LDA.components_[0]

# Top 10 words for this topic
single_topic.argsort()[-10:]

array([222, 127, 170, 231, 235,  62, 266, 290,  40, 192])

In [417]:
top_word_indices = single_topic.argsort()[-10:]
for index in top_word_indices:
    print(cv.get_feature_names_out()[index])

process
federal
like
real
reserve
checks
swift
value
banks
money


In [418]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names_out()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['bank', 'sent', 'blender', 'fruit', 'means', 'process', 'federal', 'like', 'real', 'reserve', 'checks', 'swift', 'value', 'banks', 'money']


THE TOP 15 WORDS FOR TOPIC #1
['physical', 'sent', 'check', 'say', 'started', 'll', 'fruit', 'national', '10000', 'orange', 'apple', 'new', 'waltham', 'credits', 'bank']


THE TOP 15 WORDS FOR TOPIC #2
['america', 'credits', 'deposits', 'gold', 'domain', '1000', 'moving', 'watermelon', 'different', 'farmer', 'banks', 'check', 'account', 'money', 'bank']




In [419]:
topic_results = LDA.transform(dtm)
df['topics'] = topic_results.argmax(axis=1)


In [420]:
df.to_csv(topic_path, sep=',', index=False, encoding='utf-8')